In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import codecs
import os
import gzip
import sys
from bs4 import BeautifulSoup
from itertools import chain        
from contextlib import ExitStack 
import time
import json
from sklearn.metrics import silhouette_samples
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import pymorphy2
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from multiprocessing.dummy import Lock
from gensim import corpora
from gensim.summarization import bm25
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
# from pytorch_transformers import BertTokenizer, BertConfig
# from transformers import  BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [390]:
morph = pymorphy2.MorphAnalyzer()

In [391]:
stemmerR = SnowballStemmer('russian')
stemmerE = SnowballStemmer('english')

In [392]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [393]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', 'r', 'utf8') as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [395]:
stop_words = set(stopwords.words(["russian", "english"]))

In [396]:
lock = Lock()

In [398]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [399]:
def Parser_for_titles_without_stemming(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [word for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [400]:
def Parser_for_text(ID):
    with lock:
        text_id = df[df.id == ID].text.values
    text_id = text_id[0]
    stemmed_text = [stemmerR.stem(stemmerE.stem(word)) for word in text_id.split(' ')[:512]]
    with lock:
        vector_of_text[str(ID)] = ' '.join([word for word in stemmed_text])

In [120]:
df_iter = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/data/project.csv', encoding='utf-8', chunksize=500)

In [159]:
i = 0
for df in df_iter:
    df.fillna(' ')
    vector_of_text = {}
    idx = df.id.values
    with ThreadPool(10) as pool:
        pool.map(Parser_for_text, idx)
    with open('texts/text_stemmed_part_{:05d}.json'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)
        record = json.dumps(vector_of_text, ensure_ascii=False)
        print(record, file=f_json)
    i += 1

есть 57 файлов с индексами страниц по 500

In [401]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles_without_stemming, list(range(1,28027)))

In [403]:
with open('titles_without_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

In [27]:
with open('titles_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

Застемил заголовки - уже прекрасно!

## Мутим фичи

In [2]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.fasttext import load_facebook_model

Взяли заголовки

In [3]:
vec = {}

In [4]:
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)

In [5]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [1577]:
model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')

In [6]:
len(sentences)

28026

In [7]:
first = sentences[0]
first

'м б аншин центр репродукц и генетик фертимед г москв'

In [8]:
len(model_fasttext.wv[first])

NameError: name 'model_fasttext' is not defined

Давайте попробуем сделать общий файл с эмеддингами текстов.. не просто так же их парсили

Делаем эмбединги и запоминаем их

In [40]:
filenames = os.listdir('texts_57_json')
texts_embeddings = {}
texts_embeding_to_json = {}
for f_name in filenames:
    file_n = 'texts_57_json/' + f_name
    with open(file_n, mode='r') as file:
        d = json.load(file)  # все тексты 
        for key, val in d.items():
            embedding = model_fasttext.wv[val]
            embedding_string = tuple(map(str, embedding))
            texts_embeddings[key] = embedding
            texts_embeding_to_json[key] = embedding_string

Хочу сделать Tf-Idf векторизацию

Запись в файл стринговых эмбеддингов

Попробуем прочитать эмбеддинги - не забудь перевести в float

In [9]:
with open('texts_embeddings.json', 'r', encoding='utf8') as file:
    texts_embedding_from_json = json.load(file)

In [10]:
for key, val in texts_embedding_from_json.items():
    embedding = np.array(list(map(float, texts_embedding_from_json[key])))
    texts_embedding_from_json[key] = embedding

## Фичи

Если фасттекст применять к текстам после стемминга, результат получается лучше почему-то.

Для заголовков:
1) Метки кластеров внутри группы
2) Силуэт для объекта группы

In [48]:
class Import:
    def __init__(self):
        self.doc_to_title = {}
        #self.model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')
        with open('docs_titles.tsv', encoding='utf-8') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                self.doc_to_title[doc_id] = title
        #self.top_k_cosine_titles = 3 # k ближайших косинусных расстояний для заголовков
        #self.best_docs_cnt = 10 # K самых похожих заголовков
        self.top_k_cosine_tf_idf = 10
        vec['0'] = ''
        self.doc_to_vec = TfidfVectorizer().fit_transform([vec[str(i)] for i in range(len(vec))])
        self.new_features = 10
        #self.top_k_cosine_texts = 20 # k ближайших косинусных расстояний для заголовков
    
    def make_train(self):
        train_data = pd.read_csv('train_groups.csv', encoding='utf-8')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = self.doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        svd = TruncatedSVD(n_components=5)
        y_train = []
        X_train = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:15]    )
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.zeros((train_data.shape[0], self.top_k_cosine_tf_idf + self.new_features)) # кол-во новых фичей
        groups = train_data.groupby('group_id')
        rand_index = []
        for group_id, group_indx in groups.groups.items():
            group = train_data.iloc[group_indx]
            
            docs_id = group.doc_id.values
            # сначала заберем все файлы из группы
            embedded_texts = []
            group_titles = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
                
            
            '''
            # BM25
            best_docs = []
            texts = [doc.split() for doc in group_titles]
            dictionary = corpora.Dictionary(texts)
            corpus = [dictionary.doc2bow(text) for text in texts]
            bm25_obj = bm25.BM25(corpus)
            for title in group_titles:
                query = title
                query_doc = dictionary.doc2bow(query.split())
                scores = bm25_obj.get_scores(query_doc)
                best_docs.append(sorted(range(len(scores)), key=lambda i: scores[i])[-self.best_docs_cnt:])
            '''
            #
            # embedded_titles = model_fasttext.wv[group_titles]
            tf_idf_vec = self.doc_to_vec[group.doc_id]
            svd_vec = svd.fit_transform(tf_idf_vec)
            targets = group.target.values
            model_titles = DBSCAN(0.6, metric="cosine", min_samples=8)
            labels_titles = model_titles.fit_predict(tf_idf_vec)
            '''
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=4)
            labels_texts = model_texts.fit_predict(embedded_texts)
            # добавим фичи из расстояний для эмбеддингов
            # получим матрицу из попарных расстояний
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            '''
            # добавим фичи из расстояний для tf-idf
            cos_distances = cosine_similarity(tf_idf_vec)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            tf_idf_features = sorted_dist[:, 1: self.top_k_cosine_tf_idf + 1]
            
            svd_cos_distances = cosine_similarity(svd_vec)
            svd_sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=svd_cos_distances)[:, ::-1]
            svd_features = svd_sorted_dist[:, 1: self.new_features + 1]
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_tf_idf] = tf_idf_features
            #My_X[i: i + group_indx.size, -1] = labels_titles
            My_X[i: i + group_indx.size, self.top_k_cosine_tf_idf: 
                self.top_k_cosine_tf_idf + self.new_features] = svd_features
            # валидация дала DBSCAN(0.6, metric="cosine", min_samples=8)
            #My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
        
        
        X_train = np.concatenate((X_train, My_X), axis=1)
        return X_train, y_train, groups_train
    
    def make_test(self):
        test_data = pd.read_csv('test_groups.csv', encoding='utf-8')
        testgroups_titledata = {}
        for i in range(len(test_data)):
            new_doc = test_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            title = self.doc_to_title[doc_id]
            if doc_group not in testgroups_titledata:
                testgroups_titledata[doc_group] = []
            testgroups_titledata[doc_group].append((doc_id, title))
        
        svd = TruncatedSVD(n_components=5)
        X_test = []
        groups_test = []
        for new_group in testgroups_titledata:
            docs = testgroups_titledata[new_group]
            for k, (doc_id, title) in enumerate(docs):
                groups_test.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_test.append(sorted(all_dist, reverse=True)[0:15])
        X_test = np.array(X_test)
        
        i = 0
        My_X = np.zeros((test_data.shape[0], self.top_k_cosine_tf_idf + self.new_features)) # кол-во новых фичей
        groups = test_data.groupby('group_id')
        for group_id, group_indx in groups.groups.items():
            group = test_data.iloc[group_indx]
            docs_id = group.doc_id.values
            group_titles = []
            embedded_texts = []
            for idx in docs_id:
                group_titles.append(vec[str(idx)])
                embedded_texts.append(texts_embedding_from_json[str(idx)])
                
            
            # embedded_titles = model_fasttext.wv[group_titles]
            tf_idf_vec = self.doc_to_vec[group.doc_id]
            svd_vec = svd.fit_transform(tf_idf_vec)
            model_titles = DBSCAN(0.6, metric="cosine", min_samples=8)
            labels_titles = model_titles.fit_predict(tf_idf_vec)
            '''
            if np.unique(labels_titles).size > 1:
                silhouette = silhouette_samples(embedded_titles, labels_titles)
            else:
                silhouette = np.ones(len(labels_titles)) # если всего один кластер
            model_texts = DBSCAN(0.1, metric="cosine", min_samples=4)
            labels_texts = model_texts.fit_predict(embedded_texts)
            cos_distances = cosine_similarity(embedded_titles)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            cos_features = sorted_dist[:, 1: self.top_k_cosine_titles + 1]
            '''
            # добавим фичи из расстояний для tf-idf
            cos_distances = cosine_similarity(tf_idf_vec)
            sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=cos_distances)[:, ::-1]
            tf_idf_features = sorted_dist[:, 1: self.top_k_cosine_tf_idf + 1]
            
            svd_cos_distances = cosine_similarity(svd_vec)
            svd_sorted_dist = np.apply_along_axis(np.sort, axis=1, arr=svd_cos_distances)[:, ::-1]
            svd_features = svd_sorted_dist[:, 1: self.new_features + 1]
            
            
            My_X[i: i + group_indx.size, :self.top_k_cosine_tf_idf] = tf_idf_features
#             My_X[i: i + group_indx.size, -1] = labels_titles
            My_X[i: i + group_indx.size, self.top_k_cosine_tf_idf: 
                self.top_k_cosine_tf_idf + self.new_features] = svd_features
            #My_X[i: i + group_indx.size, self.top_k_cosine_tf_idf: 
            #     self.top_k_cosine_tf_idf + self.top_k_cosine_titles] = cos_features
            #My_X[i: i + group_indx.size, :self.top_k_cosine_titles] = cos_features
            #My_X[i: i + group_indx.size, -1] = labels_texts
            i += group_indx.size
            
            #My_X[i: i + group_indx.size, self.top_k_cosine_titles:self.top_k_cosine_titles
            #    + self.top_k_cosine_texts] = cos_features_texts
            # валидация дала DBSCAN(0.6, metric="cosine", min_samples=8)
        
        X_test = np.concatenate((X_test, My_X), axis=1)
        groups_test = np.array(groups_test)
        return X_test, groups_test, test_data['pair_id'].tolist()

### Валидация для кластеризации

Валидация для заголовков дала DBSCAN(0.6, metric="cosine", min_samples=8)

In [49]:
importer = Import()

In [50]:
X_train, y_train, groups_train = importer.make_train()

In [51]:
X_test, groups_test, pair_ids = importer.make_test()

In [52]:
X_train.shape

(11690, 35)

In [53]:
X_test.shape

(16627, 35)

In [56]:
X_train[4]

array([2.        , 2.        , 2.        , 2.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.70656332, 0.6970693 , 0.62324526, 0.58965499, 0.58099242,
       0.57786371, 0.54620394, 0.51634023, 0.43289396, 0.42958136,
       0.99801683, 0.98619192, 0.9685004 , 0.93552901, 0.93526397,
       0.90359191, 0.88457749, 0.87763724, 0.81671146, 0.814454  ])

In [57]:
X_train[3]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.93613643, 0.77186271, 0.66278083, 0.64052685, 0.63320168,
       0.60131956, 0.59988497, 0.59492016, 0.58890143, 0.57642028])

## Модельки

In [66]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import catboost
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [59]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.transform(X_test)

In [1579]:
def feature_generator(X_train, X_test, num_model):
    indices = np.array([])
    length = X_train.shape[1]
    features = np.arange(length)
    np.random.shuffle(features)
    for i in range(num_model):
        ind = np.random.randint(0, length, length)
        ind = np.unique(np.append(ind, features[i * (length // num_model) : (i + 1) * (length // num_model + 1)]))
        yield X_train[:, ind], X_test[:, ind]

In [60]:
clf_e_1 = RandomForestClassifier(max_depth=10, n_estimators=150)
clf_e_2 = ExtraTreesClassifier(max_depth=15)
clf_e_4 = catboost.CatBoostClassifier()

In [61]:
clf_e_1.fit(X_train_sk, y_train)
clf_e_2.fit(X_train_sk, y_train)
clf_e_4.fit(X_train_sk, y_train)

C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Learning rate set to 0.029437
0:	learn: 0.6662263	total: 147ms	remaining: 2m 26s
1:	learn: 0.6413595	total: 206ms	remaining: 1m 42s
2:	learn: 0.6192100	total: 248ms	remaining: 1m 22s
3:	learn: 0.5997207	total: 269ms	remaining: 1m 6s
4:	learn: 0.5814900	total: 318ms	remaining: 1m 3s
5:	learn: 0.5632770	total: 350ms	remaining: 58s
6:	learn: 0.5474738	total: 392ms	remaining: 55.7s
7:	learn: 0.5342222	total: 414ms	remaining: 51.4s
8:	learn: 0.5208453	total: 431ms	remaining: 47.5s
9:	learn: 0.5078231	total: 458ms	remaining: 45.4s
10:	learn: 0.4965615	total: 515ms	remaining: 46.3s
11:	learn: 0.4854964	total: 530ms	remaining: 43.7s
12:	learn: 0.4757022	total: 555ms	remaining: 42.1s
13:	learn: 0.4656711	total: 574ms	remaining: 40.4s
14:	learn: 0.4570486	total: 596ms	remaining: 39.1s
15:	learn: 0.4503162	total: 624ms	remaining: 38.4s
16:	learn: 0.4439267	total: 640ms	remaining: 37s
17:	learn: 0.4380234	total: 690ms	remaining: 37.7s
18:	learn: 0.4321969	total: 703ms	remaining: 36.3s
19:	learn: 0

160:	learn: 0.3178744	total: 4.82s	remaining: 25.1s
161:	learn: 0.3177432	total: 4.83s	remaining: 25s
162:	learn: 0.3175754	total: 4.85s	remaining: 24.9s
163:	learn: 0.3174715	total: 4.87s	remaining: 24.8s
164:	learn: 0.3173419	total: 4.88s	remaining: 24.7s
165:	learn: 0.3172402	total: 4.89s	remaining: 24.6s
166:	learn: 0.3171088	total: 4.91s	remaining: 24.5s
167:	learn: 0.3169291	total: 4.93s	remaining: 24.4s
168:	learn: 0.3167286	total: 4.99s	remaining: 24.5s
169:	learn: 0.3166297	total: 5.01s	remaining: 24.5s
170:	learn: 0.3165688	total: 5.02s	remaining: 24.4s
171:	learn: 0.3164540	total: 5.04s	remaining: 24.3s
172:	learn: 0.3163252	total: 5.05s	remaining: 24.1s
173:	learn: 0.3162186	total: 5.07s	remaining: 24.1s
174:	learn: 0.3160959	total: 5.08s	remaining: 24s
175:	learn: 0.3159868	total: 5.12s	remaining: 24s
176:	learn: 0.3157836	total: 5.14s	remaining: 23.9s
177:	learn: 0.3157058	total: 5.15s	remaining: 23.8s
178:	learn: 0.3156219	total: 5.17s	remaining: 23.7s
179:	learn: 0.3155

327:	learn: 0.2969859	total: 8.26s	remaining: 16.9s
328:	learn: 0.2968675	total: 8.28s	remaining: 16.9s
329:	learn: 0.2967228	total: 8.29s	remaining: 16.8s
330:	learn: 0.2966201	total: 8.31s	remaining: 16.8s
331:	learn: 0.2965023	total: 8.33s	remaining: 16.8s
332:	learn: 0.2963611	total: 8.36s	remaining: 16.7s
333:	learn: 0.2962283	total: 8.38s	remaining: 16.7s
334:	learn: 0.2960758	total: 8.39s	remaining: 16.7s
335:	learn: 0.2958592	total: 8.41s	remaining: 16.6s
336:	learn: 0.2957005	total: 8.42s	remaining: 16.6s
337:	learn: 0.2955505	total: 8.44s	remaining: 16.5s
338:	learn: 0.2953345	total: 8.46s	remaining: 16.5s
339:	learn: 0.2952970	total: 8.47s	remaining: 16.4s
340:	learn: 0.2952203	total: 8.49s	remaining: 16.4s
341:	learn: 0.2951061	total: 8.51s	remaining: 16.4s
342:	learn: 0.2949749	total: 8.53s	remaining: 16.3s
343:	learn: 0.2948591	total: 8.54s	remaining: 16.3s
344:	learn: 0.2947869	total: 8.56s	remaining: 16.2s
345:	learn: 0.2946728	total: 8.57s	remaining: 16.2s
346:	learn: 

497:	learn: 0.2746038	total: 10.9s	remaining: 11s
498:	learn: 0.2745295	total: 10.9s	remaining: 11s
499:	learn: 0.2744089	total: 11s	remaining: 11s
500:	learn: 0.2743046	total: 11s	remaining: 10.9s
501:	learn: 0.2741787	total: 11s	remaining: 10.9s
502:	learn: 0.2740413	total: 11s	remaining: 10.9s
503:	learn: 0.2739526	total: 11s	remaining: 10.9s
504:	learn: 0.2738464	total: 11.1s	remaining: 10.8s
505:	learn: 0.2736685	total: 11.1s	remaining: 10.8s
506:	learn: 0.2734707	total: 11.1s	remaining: 10.8s
507:	learn: 0.2733089	total: 11.1s	remaining: 10.8s
508:	learn: 0.2732200	total: 11.1s	remaining: 10.7s
509:	learn: 0.2731168	total: 11.2s	remaining: 10.7s
510:	learn: 0.2729851	total: 11.2s	remaining: 10.7s
511:	learn: 0.2728475	total: 11.2s	remaining: 10.7s
512:	learn: 0.2727632	total: 11.2s	remaining: 10.6s
513:	learn: 0.2726177	total: 11.2s	remaining: 10.6s
514:	learn: 0.2724980	total: 11.2s	remaining: 10.6s
515:	learn: 0.2724587	total: 11.3s	remaining: 10.6s
516:	learn: 0.2723876	total:

663:	learn: 0.2558822	total: 14s	remaining: 7.09s
664:	learn: 0.2558561	total: 14s	remaining: 7.06s
665:	learn: 0.2558148	total: 14s	remaining: 7.04s
666:	learn: 0.2557519	total: 14.1s	remaining: 7.01s
667:	learn: 0.2556353	total: 14.1s	remaining: 6.99s
668:	learn: 0.2554326	total: 14.1s	remaining: 6.99s
669:	learn: 0.2553224	total: 14.1s	remaining: 6.97s
670:	learn: 0.2552024	total: 14.2s	remaining: 6.94s
671:	learn: 0.2550904	total: 14.2s	remaining: 6.92s
672:	learn: 0.2549866	total: 14.2s	remaining: 6.9s
673:	learn: 0.2548117	total: 14.2s	remaining: 6.88s
674:	learn: 0.2546442	total: 14.2s	remaining: 6.85s
675:	learn: 0.2545537	total: 14.2s	remaining: 6.83s
676:	learn: 0.2544196	total: 14.3s	remaining: 6.8s
677:	learn: 0.2542344	total: 14.3s	remaining: 6.78s
678:	learn: 0.2541067	total: 14.3s	remaining: 6.76s
679:	learn: 0.2540710	total: 14.3s	remaining: 6.74s
680:	learn: 0.2539997	total: 14.3s	remaining: 6.71s
681:	learn: 0.2539887	total: 14.4s	remaining: 6.69s
682:	learn: 0.253810

825:	learn: 0.2383071	total: 16.9s	remaining: 3.57s
826:	learn: 0.2381941	total: 17s	remaining: 3.55s
827:	learn: 0.2380772	total: 17s	remaining: 3.53s
828:	learn: 0.2379849	total: 17s	remaining: 3.52s
829:	learn: 0.2378676	total: 17.1s	remaining: 3.49s
830:	learn: 0.2378064	total: 17.1s	remaining: 3.47s
831:	learn: 0.2376908	total: 17.1s	remaining: 3.45s
832:	learn: 0.2375504	total: 17.1s	remaining: 3.43s
833:	learn: 0.2374205	total: 17.1s	remaining: 3.4s
834:	learn: 0.2373506	total: 17.1s	remaining: 3.38s
835:	learn: 0.2371954	total: 17.1s	remaining: 3.36s
836:	learn: 0.2370614	total: 17.2s	remaining: 3.34s
837:	learn: 0.2369580	total: 17.2s	remaining: 3.32s
838:	learn: 0.2368292	total: 17.2s	remaining: 3.3s
839:	learn: 0.2367026	total: 17.2s	remaining: 3.27s
840:	learn: 0.2365890	total: 17.2s	remaining: 3.25s
841:	learn: 0.2365635	total: 17.2s	remaining: 3.23s
842:	learn: 0.2364125	total: 17.2s	remaining: 3.21s
843:	learn: 0.2362874	total: 17.3s	remaining: 3.19s
844:	learn: 0.236142

990:	learn: 0.2222145	total: 19.8s	remaining: 180ms
991:	learn: 0.2222088	total: 19.8s	remaining: 160ms
992:	learn: 0.2221311	total: 19.8s	remaining: 140ms
993:	learn: 0.2220622	total: 19.8s	remaining: 120ms
994:	learn: 0.2219249	total: 19.8s	remaining: 99.7ms
995:	learn: 0.2218348	total: 19.9s	remaining: 79.8ms
996:	learn: 0.2217158	total: 19.9s	remaining: 59.8ms
997:	learn: 0.2215683	total: 19.9s	remaining: 39.9ms
998:	learn: 0.2214831	total: 19.9s	remaining: 19.9ms
999:	learn: 0.2213397	total: 19.9s	remaining: 0us


In [62]:
res = (clf_e_1.predict_proba(X_train_sk)[:, 1] + clf_e_2.predict_proba(X_train_sk)[: ,1]
+ clf_e_4.predict_proba(X_train_sk)[:, 1])/ 3

In [63]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res > th, dtype=int)))

0.7292938099389713
0.8142292490118578
0.8463269358041032
0.8672416230736604
0.8845919610231426
0.8507560815253123
0.783446307527649


In [1583]:
# for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
#     print(f1_score(y_train, np.array(res > th, dtype=int)))

0.7218831734960768
0.8022626660108214
0.8348843392714703
0.8626720183486238
0.8769444016633298
0.8344549125168237
0.7709578127829079


In [1584]:
f1_score(y_train, np.array(res > 0.5, dtype=int))

0.8769444016633298

In [67]:
votingC = VotingClassifier(estimators=[('rfc', clf_e_1), ('extc', clf_e_2), ('catboost', clf_e_4)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_sk, y_train)

NameError: name 'VotingClassifier' is not defined

In [65]:
vote = votingC.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(vote > th, dtype=int)))

NameError: name 'votingC' is not defined

In [1586]:
# vote = votingC.predict_proba(X_train_sk)[:, 1]
# for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
#     print(f1_score(y_train, np.array(vote > th, dtype=int)))

0.7215438290449192
0.7999017319739591
0.8332446808510638
0.8620146152743946
0.8755774561133354
0.8341455217610486
0.7680843329698291


In [1589]:
y_pls = np.array(votingC.predict_proba(X_test_sk)[:, 1] > 0.5, dtype=int)

In [1590]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

Для DBSCAN(0.475, samples = 7):

0.82 при 5 соседях для косинуса

0.848095724500219 при 15 соседях

0.8509769612131818 при 20 соседях

0.8545427858505797 при 25 соседях

Для DBSCAN(0.6, samples = 7):

0.8305378831694621 на 5 соседях

0.8496447730897491 на 15 соседях

0.858557902403496 на 25 соседях

Min_samples = 8 дает получше скор

model_titles = DBSCAN(0.7, metric="cosine", min_samples=8):

На 25 соседях 0.8600523712540006

model_titles = DBSCAN(0.7, metric="cosine", min_samples=7) на 25 соседях дает 0.8593886462882097

model_titles = DBSCAN(0.7, metric="cosine", min_samples=9) на 25 соседях дает 0.8584289240598229

model_titles = DBSCAN(0.65, metric="cosine", min_samples=8) на 25 соседях дает 0.8568099053168244

model_titles = DBSCAN(0.6, metric="cosine", min_samples=8) на 25 соседях дает 0.8567264247194286

##### Добавил соседей по текстам:


DBSCAN(0.7, metric="cosine", min_samples=5)

Для 3 соседей 0.8733713951105255

Для 15 соседей 0.8819013876586951

Для 25 соседей 0.8833702882483371

##### model_titles = DBSCAN(0.65, metric="cosine", min_samples=8) на 25 соседях дает 0.8568099053168244 -  взял это, но количество соседей никак не учитывается

Для 3 соседей model_texts = DBSCAN(0.05, metric="cosine", min_samples=5) дает 0.857598607888631

Для 3 соседей model_texts = DBSCAN(0.15, metric="cosine", min_samples=5) дает 0.8583466511695481 

Для 3 соседей model_texts = DBSCAN(0.2, metric="cosine", min_samples=5) дает 0.8596746077861709

Для 3 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=5) дает 0.8609368159929958

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=5) дает 0.8622248141128445 но это мне не нравится, потому что похоже на переобучение

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=6) дает 0.8609734771203731

Для 3 соседей model_texts = DBSCAN(0.4, metric="cosine", min_samples=4) дает 0.8611030055442077

##### Попрубую подключить одно косинусное расстояние для текстов, посмотрим, что будет

Для 1 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.8664050007268499

Для 2 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.8681672025723474

Для 20 соседей model_texts = DBSCAN(0.3, metric="cosine", min_samples=4) дает 0.880437999408109

Я не верю в расстояния между текстами!!!!

In [1513]:
test = (clf_e_1.predict_proba(X_test_sk)[:, 1] + clf_e_2.predict_proba(X_test_sk)[: ,1]
+ clf_e_4.predict_proba(X_test_sk)[:, 1])/ 3

In [1514]:
y_test = np.array(test > 0.5, dtype=int)

In [1515]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_test[i]) + str("\n"))

### Лучше не использовать fasttext фичи

# Уже прям боевая валидация

In [68]:
kfold = StratifiedKFold(n_splits=5)

In [69]:
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train_sk,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   44.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:  4.9min finished


0.7248156019410812

In [70]:
res_extra = ExtC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_extra > th, dtype=int)))

0.6997279765641348
0.763045540796964
0.7904626150058313
0.8030409686048148
0.8038733476790655
0.7604113977406844
0.6935332708528584


In [1615]:
# res_extra = ExtC_best.predict_proba(X_train_sk)[:, 1]
# for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
#     print(f1_score(y_train, np.array(res_extra > th, dtype=int)))

0.7316912972085386
0.8194977112458246
0.8700549818962049
0.9059112588524353
0.9059512796793093
0.8810184429574017
0.8109917209793905


In [71]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train_sk,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   54.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed: 12.4min finished


0.7239176588743648

In [72]:
res_forest = RFC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_forest > th, dtype=int)))

0.716020451640392
0.7954958227388303
0.8361300471945463
0.8592592592592592
0.8655848132271893
0.8375642940102871
0.7584953661639106


In [1617]:
# res_forest = RFC_best.predict_proba(X_train_sk)[:, 1]
# for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
#     print(f1_score(y_train, np.array(res_forest > th, dtype=int)))

0.740732543985836
0.8313403089187842
0.8862291554599248
0.9165945165945167
0.9215716251337716
0.8956283271495402
0.8236319275008713


In [73]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train_sk,y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 360 out of 360 | elapsed:  9.4min finished


0.7265828642763722

In [74]:
res_grad = GBC_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_grad > th, dtype=int)))

0.6947816826411075
0.7555342826373102
0.7779658809740851
0.7902616482440186
0.7862909367859863
0.7430102126234722
0.6759880686055183


In [1619]:
# res_grad = GBC_best.predict_proba(X_train_sk)[:, 1]
# for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
#     print(f1_score(y_train, np.array(res_grad > th, dtype=int)))

0.6821523708044752
0.7382695307812313
0.7585216142092205
0.769209315297727
0.7599632127529123
0.7222880352005415
0.6601686972820994


In [75]:
# Catboost tunning

CatBoost = catboost.CatBoostClassifier()
CB_param_grid = {'depth' : [3, 4, 5, 6],
              'l2_leaf_reg' : [21500, 22000, 25000, 28000, 28500],
              'scale_pos_weight': [1, 2, 2.4] 
              }

gsCatBoost = GridSearchCV(CatBoost,param_grid = CB_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsCatBoost.fit(X_train_sk, y_train)

CB_best = gsCatBoost.best_estimator_

# Best score
gsCatBoost.best_score_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  9.2min


KeyboardInterrupt: 

In [ ]:
CB_best = gsCatBoost.best_estimator_

In [ ]:
res_catB = CB_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_catB > th, dtype=int)))

In [1624]:
# res_catB = CB_best.predict_proba(X_train_sk)[:, 1]
# for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
#     print(f1_score(y_train, np.array(res_catB > th, dtype=int)))

0.617937049677664
0.6789383561643836
0.7002039428959891
0.7330012453300125
0.7423881387344452
0.7415062287655719
0.7136648961854177


In [76]:
import xgboost as xgb

XGB = xgb.XGBClassifier()
xgb_param_grid = {'alpha' : [0.01, 0.1],
              'n_estimators' : [10,20,30],
              'eval_metric': ['auc'],
              'max_depth': [4, 6],
              'scale_pos_weight': [1.5, 1.7, 2, 2.4] 
              }

gsXGB = GridSearchCV(XGB,param_grid = xgb_param_grid, cv=kfold, scoring="f1", n_jobs= 4, verbose = 1)

gsXGB.fit(X_train_sk,y_train)

XGB_best = gsXGB.best_estimator_

# Best score
gsXGB.best_score_

C:\Users\Maria\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   10.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   35.9s
[Parallel(n_jobs=4)]: Done 240 out of 240 | elapsed:   47.1s finished


0.7425616026285582

In [78]:
res_XGB = XGB_best.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res_XGB > th, dtype=int)))

0.6612179806638094
0.7154202514890802
0.7562093079334459
0.7730441518202943
0.7855678556785569
0.775323910482921
0.7352746525479815


In [92]:
from sklearn.ensemble import StackingClassifier

ImportError: cannot import name 'StackingClassifier' from 'sklearn.ensemble' (C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\ensemble\__init__.py)

In [ ]:
estimators = [('rf', RFC_best), ('et', ExtC_best), ('gb', GBC_best), ('cb', XGB_best)]

In [ ]:
stack = StackingClassifier(estimators=estimators, final_estimator=MLPClassifier())

In [ ]:
stack.fit(X_train_sk, y_train)

In [ ]:
stack_res = stack.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(stack_res > th, dtype=int)))

In [1651]:
# stack_res = stack.predict_proba(X_train_sk)[:, 1]
# for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
#     print(f1_score(y_train, np.array(stack_res > th, dtype=int)))

0.6913659793814433
0.7452807060554058
0.759767687434002
0.7695955369595537
0.7710879734419798
0.735483870967742
0.6928738108059593


Нужно подобрать порог!!!

In [1640]:
y_pls = np.array(stack.predict_proba(X_test_sk)[:, 1] > 0.4, dtype=int)

In [1641]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

на Кэгле = 0.68162 - сильное переобучение - решил убрать расстояние между текстами. Было 25 cosine, 0.65, 8; 0.3, 4

на Кэгле = 0.69587 без текстов но 25 cosine, 0.65, 8; 0.3, 4

На валидации 0.7809; на кэгле - 0.729. Это на 15 фичах от Севы и 10 tf-idf фичей

In [91]:
from sklearn.ensemble import VotingClassifier

In [96]:
votingC = VotingClassifier(estimators=[('rf', RFC_best), ('et', ExtC_best), ('gb', GBC_best), ('xgb', XGB_best),
                                       ('stack', stack)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train_sk, y_train)

C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [1653]:
vote = votingC.predict_proba(X_train_sk)[:, 1]
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(vote > th, dtype=int)))

0.6924516531503433
0.7530892982047096
0.7952544967470341
0.8075388636676296
0.8316212594908442
0.81343886286545
0.751706791232483


Валидация RandomForest + ExtraTree = 0.8157055925818604 на 25 фичах на titles и 3 на текстах. model_titles = DBSCAN(0.6, metric="cosine", min_samples=8); model_texts = DBSCAN(0.1, metric="cosine", min_samples=5)

In [1655]:
y_pls = np.array(votingC.predict_proba(X_test_sk)[:, 1] > 0.5, dtype=int)

In [1656]:
with open("/Users/egor/Desktop/subpobeda.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_pls)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_pls[i]) + str("\n"))

на Кэгле 0.69022 текстов но 25 cosine, 0.65, 8; 0.1, 4

У Вани на фичах без текстов на рандомфоресте и экстраТри 0.71 на кагле

У Вани на фичах с 20 текстами на рандомфоресте и экстраТри 0.706 на кагле

Три модельки на маленьком количестве фичей 0.71702

Стекинг на 25 + 20 + 2 + 25 фичей от Севы и три модели  скор = 0.6992. Наверное из-за текстов

Стекинг на маленьком количестве фичей скор = 0.71886

Без стекинга Голосование из 4 моделей: RandomForest, ExtraTrees, Градиентный бустинг, XGBoost 25 + 25 + 2 фичей  скор на кагле = 0.72434

Голосование над стекингом и еще 4 модели дает 0.72298 на 10 tf-idf фичах

Голосование из 5 моделей: Stacking, RandomForest, ExtraTrees, Градиентный бустинг, XGBoost 25 + 25 + 2 фичей скор на кагле = 0.7270

##### Стекинг над 4 моделями 7 tf-idf фичей дал 0.734

Голосование над 5 моделями включая стекинг дало 0.83 на валидации и 0.731 на кэгле. Это 15 фичей от Севы и 10 tf_idf

Стекинг без голосования дало 0.7344 на кэгле, а на валидации 0.7688 на 5 tf-idf фичах и 25 Севиных 

##### Стекинг на катбусте и еще 3 модели, final_esimator = MLPClassifier() на 15 фичах от Севы и 10 tf-idf на валидации 0.831, а на кэгле 0.73644

In [242]:
!pip install autokeras

     |████████████████████████████████| 164 kB 975 kB/s eta 0:00:01
     |████████████████████████████████| 62 kB 721 kB/s eta 0:00:01
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)


  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-py3-none-any.whl size=78936 sha256=5b8eacfd3d09dd47ffe5decadce22a2d57b62f0a46c588730b79fab7b168ed05
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/78/e2/80/7fe373cad54ad22b06d0d6204cbc29cead9e69bb2680327775
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=7c6f55f8556c75aaa3a92189474a1ffb6f2ca6f8c73e8020606f772f5a8b3dda
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=9d9d12e13a5cd421f5588c17da0e5c9c02f47e7ed7c9da558a50df822f5241d9
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built keras-tuner future terminaltables
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m 

In [243]:
import autokeras as ak

In [244]:
X_train.shape

(11690, 36)

In [245]:
y_train.shape

(11690,)

In [246]:
data = pd.DataFrame()

In [253]:
y = np.array(y_train)[:,np.newaxis]
y.shape

(11690, 1)

In [257]:
Train_df = pd.DataFrame(data=X_y)

In [258]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11690 entries, 0 to 11689
Data columns (total 37 columns):
0     11690 non-null float64
1     11690 non-null float64
2     11690 non-null float64
3     11690 non-null float64
4     11690 non-null float64
5     11690 non-null float64
6     11690 non-null float64
7     11690 non-null float64
8     11690 non-null float64
9     11690 non-null float64
10    11690 non-null float64
11    11690 non-null float64
12    11690 non-null float64
13    11690 non-null float64
14    11690 non-null float64
15    11690 non-null float64
16    11690 non-null float64
17    11690 non-null float64
18    11690 non-null float64
19    11690 non-null float64
20    11690 non-null float64
21    11690 non-null float64
22    11690 non-null float64
23    11690 non-null float64
24    11690 non-null float64
25    11690 non-null float64
26    11690 non-null float64
27    11690 non-null float64
28    11690 non-null float64
29    11690 non-null float64
30    11690 non-null 

In [259]:
Train_df.to_csv('train_df.csv')

In [263]:
Train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,4.0,4.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,...,0.614383,0.528873,0.509626,0.504905,0.501869,0.461926,0.449864,-1.0,0.0,0.0
1,8.0,7.0,7.0,7.0,7.0,7.0,4.0,4.0,3.0,3.0,...,0.653389,0.628512,0.580146,0.533492,0.457677,0.426910,0.407499,0.0,0.0,0.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.282926,0.263492,0.260648,0.248446,0.230717,0.230373,0.228871,-1.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.188717,0.179073,0.151745,0.146303,0.135194,0.132056,0.129261,-1.0,0.0,0.0
4,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.450056,0.424316,0.422547,0.414055,0.398026,0.359367,0.339476,-1.0,0.0,0.0


In [266]:
Train_df[36]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       1.0
29       0.0
        ... 
11660    0.0
11661    0.0
11662    0.0
11663    1.0
11664    1.0
11665    0.0
11666    0.0
11667    0.0
11668    0.0
11669    0.0
11670    1.0
11671    1.0
11672    0.0
11673    0.0
11674    0.0
11675    0.0
11676    0.0
11677    0.0
11678    0.0
11679    0.0
11680    0.0
11681    0.0
11682    0.0
11683    0.0
11684    0.0
11685    0.0
11686    0.0
11687    0.0
11688    0.0
11689    0.0
Name: 36, Length: 11690, dtype: float64

In [338]:
clf = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=5)

In [339]:
clf.fit(X_train, y_train, epochs=20)

Trial 5 Complete [00h 00m 20s]
val_accuracy: 0.8517718315124512

Best val_accuracy So Far: 0.8517718315124512
Total elapsed time: 00h 01m 27s
INFO:tensorflow:Oracle triggered exit

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/20
366/366 [==============================] - 1s 2ms/step - loss: 0.6339 - accuracy: 0.6887
Epoch 2/20
366/366 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7407
Epoch 3/20
366/366 [==============================] - 1s 2ms/step - loss: 0.4903 - accuracy: 0.7815
Epoch 4/20
366/366 [========

In [340]:
X_test.shape

(16627, 36)

In [341]:
predicted_y = clf.predict(X_test)

In [342]:
predicted_y_train = clf.predict(X_train)

In [343]:
f1_score(predicted_y_train, y_train)

0.7126715092816788

In [344]:
predicted_y = predicted_y.reshape(-1, 1)

In [345]:
predicted_y = predicted_y.astype(int)

In [ ]:
predicted_y = np.array(predicted_y).ravel()

In [333]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(predicted_y[i]) + str("\n"))